# Setup: Generate Sample Dataset

This cell creates the required folder structure (`data/raw/` and `data/processed/`) relative to the notebook, and generates the sample CSV dataset with missing values. 
This ensures the dataset is ready for cleaning functions and saves it to `data/raw/sample_data.csv`.

In [3]:
import os
import pandas as pd
import numpy as np

# Define folder paths relative to this notebook
raw_dir = '../data/raw'
processed_dir = '../data/processed'

# Create folders if they don't exist
os.makedirs(raw_dir, exist_ok=True)
os.makedirs(processed_dir, exist_ok=True)

# Define the sample data
data = {
    'age': [34, 45, 29, 50, 38, np.nan, 41],
    'income': [55000, np.nan, 42000, 58000, np.nan, np.nan, 49000],
    'score': [0.82, 0.91, np.nan, 0.76, 0.88, 0.65, 0.79],
    'zipcode': ['90210', '10001', '60614', '94103', '73301', '12345', '94105'],
    'city': ['Beverly', 'New York', 'Chicago', 'SF', 'Austin', 'Unknown', 'San Francisco'],
    'extra_data': [np.nan, 42, np.nan, np.nan, np.nan, 5, np.nan]
}

# Create DataFrame
df = pd.DataFrame(data)

# Save to CSV in raw data folder
csv_path = os.path.join(raw_dir, 'sample_data.csv')
if not os.path.exists(csv_path):
    df.to_csv(csv_path, index=False)
    print(f'Sample dataset created and saved to {csv_path}')
else:
    print(f'File already exists at {csv_path}. Skipping CSV creation to avoid overwrite.')


Sample dataset created and saved to data/raw/sample_data.csv


# Homework Starter — Stage 6: Data Preprocessing
Use this notebook to apply your cleaning functions and save processed data.

In [8]:
import pandas as pd
from src import cleaning

## Load Raw Dataset

In [20]:
df = pd.read_csv('../data/raw/sample_data.csv')
df

,age,income,score,zipcode,city,extra_data
0,34.0,55000.0,0.82,90210,Beverly,NaN
1,45.0,NaN,0.91,10001,New York,42.0
2,29.0,42000.0,NaN,60614,Chicago,NaN
3,50.0,58000.0,0.76,94103,SF,NaN
4,38.0,NaN,0.88,73301,Austin,NaN
5,NaN,NaN,0.65,12345,Unknown,5.0
6,41.0,49000.0,0.79,94105,San Francisco,NaN


## Apply Cleaning Functions

In [23]:
# Apply functions
df_clean = cleaning.drop_missing(df, threshold=0.5)
df_clean = cleaning.fill_missing_median(df_clean, ['age','income', 'score'])
df_clean = cleaning.normalize_data(df_clean, ['score'])

# Preview
df_clean

,age,income,score,zipcode,city,extra_data
0,34.0,55000.0,0.653846,90210,Beverly,NaN
1,45.0,52000.0,1.000000,10001,New York,42.0
2,29.0,42000.0,0.596154,60614,Chicago,NaN
3,50.0,58000.0,0.423077,94103,SF,NaN
4,38.0,52000.0,0.884615,73301,Austin,NaN
5,39.5,52000.0,0.000000,12345,Unknown,5.0
6,41.0,49000.0,0.538462,94105,San Francisco,NaN


## Save Cleaned Dataset

In [24]:
df_clean.to_csv('../data/processed/sample_data_cleaned.csv', index=False)

## Compare Original vs Cleaned datasets

In [29]:
df.info()
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   age         6 non-null      float64
 1   income      4 non-null      float64
 2   score       6 non-null      float64
 3   zipcode     7 non-null      int64  
 4   city        7 non-null      object 
 5   extra_data  2 non-null      float64
dtypes: float64(4), int64(1), object(1)
memory usage: 468.0+ bytes
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   age         7 non-null      float64
 1   income      7 non-null      float64
 2   score       7 non-null      float64
 3   zipcode     7 non-null      int64  
 4   city        7 non-null      object 
 5   extra_data  2 non-null      float64
dtypes: float64(4), int64(1), object(1)
memory usage: 468.0+ bytes


## Assumptions & Notes

**Missingness policy**
  - Rows with >50% missing (of remaining cols) are dropped
  - Numeric features are imputed with **median** (robust to outliers); Categorical/text left as-is

**Feature selection for scaling**
  - Only true **numeric feature** columns are scaled; ID-like (e.g.`zip`) and binary/constant columns are **not** scaled

**Normalization**
  - Used **min–max** scaling to [0, 1] (or z-score if specified). This is done after imputation because scalers do not accept NaNs
    
**Reproducibility**
  - Cleaned data saved to `data/processed/` as csv
